In [ ]:
# This cell is only for setting example parameter defaults - gets replaced by sidecar.
day_obs = "20250723"
timezone = "Chile/Continental"
#timezone = "UTC"

# Almanac for {{ params.day_obs }} 

* [Almanac](#Almanac)


In [ ]:
import os
import warnings
from math import floor
import numpy as np
import healpy as hp
import matplotlib.pylab as plt
from cycler import cycler
import colorcet as cc

import pandas as pd
from pandas import option_context
from IPython.display import display, Markdown, HTML

import datetime
from astropy.time import Time, TimeDelta
import astropy.units as u
import astropy
astropy.utils.iers.conf.iers_degraded_accuracy = 'ignore'

import datetime
import pytz

import datetime
from zoneinfo import ZoneInfo

try:
    tz = ZoneInfo(timezone)
    tz_utc = ZoneInfo("UTC")
except ZoneInfoNotFoundError:
    print("Timezone should be a string recognizable to `ZoneInfo`.")
    print("Using Chile/Continental (+UTC) backup.")
    tz = ZoneInfo("Chile/Continental")
    tz_utc = ZoneInfo("UTC")

from rubin_scheduler.site_models import Almanac
from rubin_scheduler.utils import Site

if 'usdf' in os.getenv("EXTERNAL_INSTANCE_URL", ""):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
str_day_obs = f"{str(day_obs)[0:4]}-{str(day_obs)[4:6]}-{str(day_obs)[6:]}"
day_obs_time = Time(f"{str_day_obs}T12:00:00", format='isot', scale='tai')

t_start = Time(f"{str_day_obs}T12:00:00", format='isot', scale='tai')
t_end = Time(f"{str_day_obs}T12:00:00", format='isot', scale='tai') + TimeDelta(1, format='jd')

<a id="Almanac"></a>

In [ ]:
## Almanac ## 

display(Markdown(f"## Almanac information for dayobs {day_obs}"))
site = Site('LSST')
almanac = Almanac()
night_events = almanac.get_sunset_info(evening_date=day_obs, longitude=site.longitude_rad)
sunset_12 = Time(night_events['sun_n12_setting'], format='mjd', scale='utc') 
sunrise_12 = Time(night_events['sun_n12_rising'], format='mjd', scale='utc')
sunset = Time(night_events['sunset'], format='mjd', scale='utc') 
sunrise = Time(night_events['sunrise'], format='mjd', scale='utc')
night_length = sunrise_12.mjd - sunset_12.mjd

display(Markdown(f"0-deg sunset at {sunset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC  -- {sunset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"0-deg sunrise at {sunrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC --  {sunrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"12-deg sunset at {sunset_12.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC  -- {sunset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"12-deg sunrise at {sunrise_12.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC --  {sunrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"allowing for a (-12deg) night of {night_length * 24 :.2f} hours"))
moon_phase = almanac.get_sun_moon_positions(sunset_12.mjd)['moon_phase']
if not np.isnan(night_events['moonrise']):
    moonrise = Time(night_events['moonrise'], format='mjd', scale='utc')
    display(Markdown(f"Moonrise is at {moonrise.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonrise.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
if not np.isnan(night_events['moonset']):
    moonset = Time(night_events['moonset'], format='mjd', scale='utc')
    display(Markdown(f"Moonset at {moonset.to_datetime(timezone=tz_utc).strftime('%x %X')} UTC -- {moonset.to_datetime(timezone=tz).strftime('%x %X')} {timezone}"))
display(Markdown(f"Moon phase is {moon_phase :.1f} (0=new, 100=full)."))